In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib
import copy

In [ ]:
# for fonts of the plots
matplotlib.rc("font", **{"family":"serif","serif":["Computer Modern Roman"]})
matplotlib.rc("text", usetex=True)

## Experimental data

In [ ]:
# experiment parameters

Lx = 2000
Ly = 2000
pixtomm = 7.8/900
xnum = 200
ynum = 200
dx = pixtomm*Lx/xnum
dy = pixtomm*Ly/ynum
n_j = 400
wnum = int((n_j-1)/2)+1
dkx = 2*np.pi/(Lx*pixtomm)
dky = 2*np.pi/(Ly*pixtomm)
dt = 10
dw = 2*np.pi/(dt*n_j)

# load current correlation files
# experimental data
# after excluding the zeros in the data due to losing of the embryos.

N = 730

with open('Cll_expt.dat','rb') as f:
    Cll = np.fromfile(f,np.float64)
Cll = (1/N)*Cll.reshape((xnum,ynum,wnum))

with open('Cltr_expt.dat','rb') as f:
    Cltr = np.fromfile(f,np.float64)
Cltr = (1/N)*Cltr.reshape((xnum,ynum,wnum))

with open('Clti_expt.dat','rb') as f:
    Clti = np.fromfile(f,np.float64)
Clti = (1/N)*Clti.reshape((xnum,ynum,wnum))

with open('Ctt_expt.dat','rb') as f:
    Ctt = np.fromfile(f,np.float64)
Ctt = (1/N)*Ctt.reshape((xnum,ynum,wnum))

Cfull = copy.copy(Cll)+copy.copy(Ctt)

In [ ]:
# frequency domain
kxrange = np.zeros(xnum)
kyrange = np.zeros(ynum)
wrange = dw*np.arange(wnum)

for i in range(xnum):
    kxrange[i] = -np.pi/dx + dkx*i
for j in range(ynum):
    kyrange[j] = -np.pi/dy + dky*j
    
kx,ky = np.meshgrid(kxrange,kyrange,indexing='ij')

## Analysis

In [ ]:
# 2d matrix whose elements are the max value of C at particular (qx,qy)
# preserve the sign of Cltr and Clti

Cll_copy = copy.copy(Cll)
Cltr_copy = copy.copy(Cltr)
Clti_copy = copy.copy(Clti)
Ctt_copy = copy.copy(Ctt)
Cfull_copy = copy.copy(Cfull)

Cll_peak = np.zeros([xnum,ynum])
Cltr_peak = np.zeros([xnum,ynum])
Clti_peak = np.zeros([xnum,ynum])
Ctt_peak = np.zeros([xnum,ynum])

Cfull_peak = np.zeros([xnum,ynum])

wll_peak = np.zeros([xnum,ynum])
wltr_peak = np.zeros([xnum,ynum])
wlti_peak = np.zeros([xnum,ynum])
wtt_peak = np.zeros([xnum,ynum])

wfull_peak = np.zeros([xnum,ynum])

fromhere = 1

for i in range(xnum):
    for j in range(ynum):
        Cll_thisq = Cll_copy[i,j,fromhere:]
        Ctt_thisq = Ctt_copy[i,j,fromhere:]
        Cltr_thisq = Cltr_copy[i,j,fromhere:]
        Clti_thisq = Clti_copy[i,j,fromhere:]
        Cltr_thisq_pos = abs(Cltr_copy[i,j,fromhere:])
        Clti_thisq_pos = abs(Clti_copy[i,j,fromhere:])
        Cfull_thisq = Cfull_copy[i,j,fromhere:]
        
        Cll_peak[i,j] = np.max(Cll_thisq) # record max C value at certain q
        idx_wllmax = np.where(Cll_thisq==np.max(Cll_thisq)) # index in w dimension for the max C
        idx_wllmax = int(idx_wllmax[0][0]) + fromhere # make the index an integer and add fromhere
        wll_peak[i,j] = wrange[idx_wllmax] # record the value of max peak w. 
        
        Ctt_peak[i,j] = np.max(Ctt_thisq) # record max C value at certain q
        idx_wttmax = np.where(Ctt_thisq==np.max(Ctt_thisq)) # index in w dimension for the max C
        idx_wttmax = int(idx_wttmax[0][0]) + fromhere # make the index an integer and add fromhere
        wtt_peak[i,j] = wrange[idx_wttmax] # record the value of max peak w. 
        
        idx_wltrmax = np.where(Cltr_thisq_pos==np.max(Cltr_thisq_pos))
        idx_wltrmax = int(idx_wltrmax[0][0]) + fromhere
        wltr_peak[i,j] = wrange[idx_wltrmax]
        Cltr_peak[i,j] = Cltr_thisq[idx_wltrmax - fromhere]
        
        idx_wltimax = np.where(Clti_thisq_pos==np.max(Clti_thisq_pos))
        idx_wltimax = int(idx_wltimax[0][0]) + fromhere
        wlti_peak[i,j] = wrange[idx_wltimax]
        Clti_peak[i,j] = Clti_thisq[idx_wltimax - fromhere]
        
        Cfull_peak[i,j] = np.max(Cfull_thisq)
        idx_wfullmax = np.where(Cfull_thisq==np.max(Cfull_thisq))
        idx_wfullmax = int(idx_wfullmax[0][0]) + fromhere
        wfull_peak[i,j] = wrange[idx_wfullmax]
        

## Dispersion relations from the experimental data

In [ ]:
# Dispersion plot.

C_plot = copy.copy(Cfull_peak)
w_plot = copy.copy(wfull_peak)

peak_thres = 0.075

kx_thres = kx[np.abs(C_plot)>peak_thres]
ky_thres = ky[np.abs(C_plot)>peak_thres]
wpk_thres = w_plot[np.abs(C_plot)>peak_thres]

#%matplotlib notebook
%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(12,10))
ax = fig.add_subplot(111,projection='3d')

img = ax.scatter3D(kx_thres,ky_thres,wpk_thres,c=C_plot[C_plot>peak_thres],cmap = 'viridis',rasterized=True)

ax.plot([0,0],[0,0],[0,0.3],'r--')
ax.plot([0,0],[-30,40],[0.03,0.03],'r--')
ax.plot([0,45],[40,40],[0.03,0.03],'r--')
ax.plot(0,0,0,'ro')

ax.set_zlim([0,0.3])
ax.set_xlim([-38,38])
ax.set_ylim([-38,38])

cb = ax.figure.colorbar(img,shrink=0.7,pad=0.1)
ax.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)

cb.set_label('C',fontsize=20)
ax.tick_params(labelsize=15)
cb.ax.tick_params(labelsize=15)
ax.text2D(0.09,-0.022,'0.03',fontsize=20,color='red')
#plt.savefig('expt_dispersion_v8_thres0015.png',dpi=200,bbox_inches='tight')
plt.show()


In [ ]:
# get the data for 2d projection of the 3d dispersion plot
C_qx0 = copy.copy(Cfull)
C_qx0 = C_qx0[100,:,:] # slice at qy = 0

q_slice,w_slice = np.meshgrid(kyrange,wrange,indexing='ij')

fromthisw = 1 # not include w=0
uptothisw = -1 # high frequency w is not relevant
w_slice = w_slice[:,fromthisw:uptothisw] # only take w>0.
q_slice = q_slice[:,fromthisw:uptothisw]
C_slice = C_qx0[:,fromthisw:uptothisw]

q_slice_zoom = q_slice[:,:40]
w_slice_zoom = w_slice[:,:40]
C_slice_zoom = C_slice[:,:40]


# plotting
C_plot = copy.copy(Cfull_peak)
w_plot = copy.copy(wfull_peak)

peak_thres = 0.075

kx_thres = kx[np.abs(C_plot)>peak_thres]
ky_thres = ky[np.abs(C_plot)>peak_thres]
wpk_thres = w_plot[np.abs(C_plot)>peak_thres]

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(10,12))
ax1 = fig.add_subplot(3,2,(1,4),projection='3d')
ax2 = fig.add_subplot(3,2,5)
ax3 = fig.add_subplot(3,2,6)

# first plot
img1 = ax1.scatter3D(kx_thres,ky_thres,wpk_thres,c=C_plot[C_plot>peak_thres],cmap = 'viridis',rasterized=True)

ax1.plot([0,0],[0,0],[0,0.3],'r--')
ax1.plot([0,0],[-30,40],[0.03,0.03],'r--')
ax1.plot([0,45],[40,40],[0.03,0.03],'r--')
ax1.plot(0,0,0,'ro')

ax1.set_zlim([0,0.3])
ax1.set_xlim([-38,38])
ax1.set_ylim([-38,38])

cb1 = ax1.figure.colorbar(img1,shrink=0.7,pad=0.1)
ax1.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)

cb1.set_label('C',fontsize=20)
ax1.tick_params(labelsize=15)
cb1.ax.tick_params(labelsize=15)
ax1.text2D(0.09,-0.024,'0.03',fontsize=20,color='red')
ax1.text2D(-0.1,0.07,'(a)',fontsize=25,va='top',ha='right')

# second plot
img2 = ax2.scatter(kx_thres,ky_thres,c=C_plot[C_plot>peak_thres],cmap='viridis',rasterized=True)

ax2.plot([p1x,p2x,p3x,p4x,p5x,p6x,p1x],[p1y,p2y,p3y,p4y,p5y,p6y,p1y],'k')
ax2.plot([bz1x,bz2x,bz3x,bz4x,bz5x,bz6x,bz1x],[bz1y,bz2y,bz3y,bz4y,bz5y,bz6y,bz1y],'k')
ax2.plot([0,0],[-36,35],'r--')

cb2 = fig.colorbar(img2,ax=ax2,shrink=0.85,ticks=[0.0,0.5,1.0])
ax2.set_xlabel('$q_x\ \\rm{(mm^{-1})}$',fontsize=20)
ax2.set_ylabel('$q_y\ \\rm{(mm^{-1})}$',fontsize=20)
cb2.set_label('C',fontsize=20)
ax2.tick_params(labelsize=15)
cb2.ax.tick_params(labelsize=15)
ax2.set_aspect('equal')
ax2.text(-42,45,'(b)',fontsize=25,va='top',ha='right')

# third plot
img3 = ax3.scatter(q_slice_zoom[np.abs(C_slice_zoom)>peak_thres],w_slice_zoom[np.abs(C_slice_zoom)>peak_thres],c=C_slice_zoom[np.abs(C_slice_zoom)>peak_thres],cmap='viridis',rasterized=True)

cb3 = fig.colorbar(img3,ax=ax3,shrink=0.85)
ax3.set_xlabel('$q_y\ \\rm{(mm^{-1})}$',fontsize=20)
ax3.set_ylabel('$\omega\ \\rm{(rad/s)}$',fontsize=20)
cb3.set_label('C',fontsize=20)
ax3.tick_params(labelsize=15)
cb3.ax.tick_params(labelsize=15)
ax3.set_box_aspect(aspect=1)
ax3.text(-45,0.0655,'(c)',fontsize=25,va='top',ha='right')
ax3.set_xlim([-30,30])
ax3.set_ylim([0,0.06])
    
fig.tight_layout()

#plt.savefig('expt_dispersion_v11_thres0075.pdf',dpi=200,bbox_inches='tight')
plt.show()


## Dispersion plot before and after applying the threshold

In [ ]:
C_plot = copy.copy(Cfull_peak)
w_plot = copy.copy(wfull_peak)

peak_thres = 0
peak_thres2 = 0.075

kx1 = kx[np.abs(C_plot)>peak_thres]
ky1 = ky[np.abs(C_plot)>peak_thres]
wpk1 = w_plot[np.abs(C_plot)>peak_thres]

kx2 = kx[np.abs(C_plot)>peak_thres2]
ky2 = ky[np.abs(C_plot)>peak_thres2]
wpk2 = w_plot[np.abs(C_plot)>peak_thres2]

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(23,7))
ax1 = fig.add_subplot(1,2,1,projection='3d')
ax2 = fig.add_subplot(1,2,2,projection='3d')
fig.tight_layout()

# first plot no threshold
img1 = ax1.scatter3D(kx1,ky1,wpk1,c=C_plot[np.abs(C_plot)>peak_thres],cmap = 'viridis',s=30,rasterized=True)

cb1 = ax1.figure.colorbar(img1,shrink=0.7,pad=0.08)
ax1.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb1.set_label('$C$',fontsize=20)

ax1.text2D(-0.08,0.06,'(a)',fontsize=25)
ax1.tick_params(labelsize=15)
cb1.ax.tick_params(labelsize=15)

# second plot with threshold
img2 = ax2.scatter3D(kx2,ky2,wpk2,c=C_plot[np.abs(C_plot)>peak_thres2],cmap = 'viridis',s=30,rasterized=True)

ax2.plot([0,0],[0,0],[0,0.3],'r--')
ax2.plot([0,0],[-30,40],[0.03,0.03],'r--')
ax2.plot([0,45],[40,40],[0.03,0.03],'r--')
ax2.plot(0,0,0,'ro')

ax2.set_zlim([0,0.3])
ax2.set_xlim([-38,38])
ax2.set_ylim([-38,38])
ax2.text2D(0.09,-0.022,'0.03',fontsize=20,color='red')

cb2 = ax2.figure.colorbar(img2,shrink=0.7,pad=0.08)
ax2.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax2.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax2.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb2.set_label('$C$',fontsize=20)

ax2.text2D(-0.08,0.06,'(b)',fontsize=25)
ax2.tick_params(labelsize=15)
cb2.ax.tick_params(labelsize=15)

#plt.savefig('expt_Cfull_thres_v2.pdf',dpi=200,bbox_inches='tight')
plt.show()


## Dispersion from each element of the current correlation tensor without threshold

In [ ]:
Cll_plot = copy.copy(Cll_peak)
wll_plot = copy.copy(wll_peak)
Ctt_plot = copy.copy(Ctt_peak)
wtt_plot = copy.copy(wtt_peak)
Cltr_plot = copy.copy(Cltr_peak)
wltr_plot = copy.copy(wltr_peak)
Clti_plot = copy.copy(Clti_peak)
wlti_plot = copy.copy(wlti_peak)

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(23,15))
ax1 = fig.add_subplot(2,2,1,projection='3d')
ax2 = fig.add_subplot(2,2,2,projection='3d')
ax3 = fig.add_subplot(2,2,3,projection='3d')
ax4 = fig.add_subplot(2,2,4,projection='3d')
fig.tight_layout()

# first plot Cll
img1 = ax1.scatter3D(kx,ky,wll_plot,c=Cll_plot,cmap = 'viridis',s=20,rasterized=True)

cb1 = ax1.figure.colorbar(img1,shrink=0.7,pad=0.08)
ax1.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb1.set_label('$C$',fontsize=20)

ax1.text2D(-0.08,0.06,'(a)',fontsize=25)
ax1.tick_params(labelsize=15)
cb1.ax.tick_params(labelsize=15)

# second plot Ctt
img2 = ax2.scatter3D(kx,ky,wtt_plot,c=Ctt_plot,cmap = 'viridis',s=20,rasterized=True)

cb2 = ax2.figure.colorbar(img2,shrink=0.7,pad=0.08)
ax2.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax2.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax2.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb2.set_label('$C$',fontsize=20)

ax2.text2D(-0.08,0.06,'(b)',fontsize=25)
ax2.tick_params(labelsize=15)
cb2.ax.tick_params(labelsize=15)

# third plot Cltr
img3 = ax3.scatter3D(kx,ky,wltr_plot,c=Cltr_plot,cmap = 'viridis',s=20,rasterized=True)

cb3 = ax3.figure.colorbar(img3,shrink=0.7,pad=0.08)
ax3.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax3.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax3.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb3.set_label('$C$',fontsize=20)

ax3.text2D(-0.08,0.06,'(c)',fontsize=25)
ax3.tick_params(labelsize=15)
cb3.ax.tick_params(labelsize=15)

# fourth plot Clti
img4 = ax4.scatter3D(kx,ky,wlti_plot,c=Clti_plot,cmap = 'viridis',s=20,rasterized=True)

cb4 = ax4.figure.colorbar(img4,shrink=0.7,pad=0.08)
ax4.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax4.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax4.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb4.set_label('$C$',fontsize=20)

ax4.text2D(-0.08,0.06,'(d)',fontsize=25)
ax4.tick_params(labelsize=15)
cb4.ax.tick_params(labelsize=15)

#plt.savefig('expt_Cab_test.pdf',dpi=200,bbox_inches='tight')
plt.show()


## Dispersion from each element of current correlation tensor with threshold

In [ ]:
Cll_plot = copy.copy(Cll_peak)
wll_plot = copy.copy(wll_peak)
Ctt_plot = copy.copy(Ctt_peak)
wtt_plot = copy.copy(wtt_peak)
Cltr_plot = copy.copy(Cltr_peak)
wltr_plot = copy.copy(wltr_peak)
Clti_plot = copy.copy(Clti_peak)
wlti_plot = copy.copy(wlti_peak)

peak_thres = 0.01

kx_ll = kx[np.abs(Cll_plot)>peak_thres]
ky_ll = ky[np.abs(Cll_plot)>peak_thres]
wpk_ll = wll_plot[np.abs(Cll_plot)>peak_thres]

kx_tt = kx[np.abs(Ctt_plot)>peak_thres]
ky_tt = ky[np.abs(Ctt_plot)>peak_thres]
wpk_tt = wtt_plot[np.abs(Ctt_plot)>peak_thres]

kx_ltr = kx[np.abs(Cltr_plot)>peak_thres]
ky_ltr = ky[np.abs(Cltr_plot)>peak_thres]
wpk_ltr = wltr_plot[np.abs(Cltr_plot)>peak_thres]

kx_lti = kx[np.abs(Clti_plot)>peak_thres]
ky_lti = ky[np.abs(Clti_plot)>peak_thres]
wpk_lti = wlti_plot[np.abs(Clti_plot)>peak_thres]

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(23,15))
ax1 = fig.add_subplot(2,2,1,projection='3d')
ax2 = fig.add_subplot(2,2,2,projection='3d')
ax3 = fig.add_subplot(2,2,3,projection='3d')
ax4 = fig.add_subplot(2,2,4,projection='3d')
fig.tight_layout()

# first plot Cll
img1 = ax1.scatter3D(kx_ll,ky_ll,wpk_ll,c=Cll_plot[np.abs(Cll_plot)>peak_thres],cmap = 'viridis',s=30,rasterized=True)

ax1.plot([0,0],[0,0],[0,0.3],'r--')
ax1.plot([0,0],[-30,40],[0.03,0.03],'r--')
ax1.plot([0,45],[40,40],[0.03,0.03],'r--')
ax1.plot(0,0,0,'ro')

ax1.text2D(0.09,-0.022,'0.03',fontsize=20,color='red')

ax1.set_zlim([0,0.3])
ax1.set_xlim([-38,38])
ax1.set_ylim([-38,38])

cb1 = ax1.figure.colorbar(img1,shrink=0.7,pad=0.08)
ax1.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax1.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb1.set_label('$C$',fontsize=20)

ax1.text2D(-0.08,0.06,'(a)',fontsize=25)
ax1.tick_params(labelsize=15)
cb1.ax.tick_params(labelsize=15)

# second plot Ctt
img2 = ax2.scatter3D(kx_tt,ky_tt,wpk_tt,c=Ctt_plot[np.abs(Ctt_plot)>peak_thres],cmap = 'viridis',s=30,rasterized=True)

ax2.plot([0,0],[0,0],[0,0.3],'r--')
ax2.plot([0,0],[-30,40],[0.03,0.03],'r--')
ax2.plot([0,45],[40,40],[0.03,0.03],'r--')
ax2.plot(0,0,0,'ro')

ax2.text2D(0.09,-0.022,'0.03',fontsize=20,color='red')

ax2.set_zlim([0,0.3])
ax2.set_xlim([-38,38])
ax2.set_ylim([-38,38])

cb2 = ax2.figure.colorbar(img2,shrink=0.7,pad=0.08)
ax2.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax2.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax2.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb2.set_label('$C$',fontsize=20)

ax2.text2D(-0.08,0.06,'(b)',fontsize=25)
ax2.tick_params(labelsize=15)
cb2.ax.tick_params(labelsize=15)

# third plot Cltr
img3 = ax3.scatter3D(kx_ltr,ky_ltr,wpk_ltr,c=Cltr_plot[np.abs(Cltr_plot)>peak_thres],cmap = 'viridis',s=30,rasterized=True)

ax3.plot([0,0],[0,0],[0,0.3],'r--')
ax3.plot([0,0],[-30,40],[0.03,0.03],'r--')
ax3.plot([0,45],[40,40],[0.03,0.03],'r--')
ax3.plot(0,0,0,'ro')

ax3.text2D(0.09,-0.022,'0.03',fontsize=20,color='red')

ax3.set_zlim([0,0.3])
ax3.set_xlim([-38,38])
ax3.set_ylim([-38,38])

cb3 = ax3.figure.colorbar(img3,shrink=0.7,pad=0.08)
ax3.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax3.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax3.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb3.set_label('$C$',fontsize=20)

ax3.text2D(-0.08,0.06,'(c)',fontsize=25)
ax3.tick_params(labelsize=15)
cb3.ax.tick_params(labelsize=15)

# fourth plot Clti
img4 = ax4.scatter3D(kx_lti,ky_lti,wpk_lti,c=Clti_plot[np.abs(Clti_plot)>peak_thres],cmap = 'viridis',s=30,rasterized=True)

ax4.plot([0,0],[0,0],[0,0.3],'r--')
ax4.plot([0,0],[-30,40],[0.03,0.03],'r--')
ax4.plot([0,45],[40,40],[0.03,0.03],'r--')
ax4.plot(0,0,0,'ro')

ax4.text2D(0.09,-0.022,'0.03',fontsize=20,color='red')

ax4.set_zlim([0,0.3])
ax4.set_xlim([-38,38])
ax4.set_ylim([-38,38])

cb4 = ax4.figure.colorbar(img4,shrink=0.7,pad=0.08)
ax4.set_xlabel('$q_x\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax4.set_ylabel('$q_y\ \\rm{(mm^{-1})}$', fontsize=20, labelpad=10)
ax4.set_zlabel('$\omega\ \\rm{(rad/s)}$', fontsize=20, labelpad=10)
cb4.set_label('$C$',fontsize=20)

ax4.text2D(-0.08,0.06,'(d)',fontsize=25)
ax4.tick_params(labelsize=15)
cb4.ax.tick_params(labelsize=15)

#plt.savefig('expt_Cab_test_thres001.pdf',dpi=200,bbox_inches='tight')
plt.show()


## Comparison of experimental dispersion before and after cutting off the self-circling signal

In [ ]:
# find the path Gamma -> K -> M -> Gamma
# indices for Gamma, K, M points in the first BZ

Gpoint = np.array([100,100],dtype=int)
Mpoint = np.array([132,132],dtype=int)
Kpoint = np.array([114,151],dtype=int)

# indices for the path G->K->M->G

idisGK = Kpoint[0]-Gpoint[0]
jdisGK = Kpoint[1]-Gpoint[1]
numGtoK = max(np.abs(idisGK),np.abs(jdisGK))
GtoK = np.zeros([2,numGtoK],dtype = int)
GtoK[0,0]=Gpoint[0]
GtoK[1,0]=Gpoint[1]
if np.abs(idisGK)>np.abs(jdisGK):
    idisGKsign = idisGK/np.abs(idisGK)
    for i in range(1,numGtoK):
        GtoK[0,i]=GtoK[0,0]+i*idisGKsign
        GtoK[1,i]=int(GtoK[1,0]+i*idisGKsign*jdisGK/idisGK)
else:
    jdisGKsign = jdisGK/np.abs(jdisGK)
    for i in range(1,numGtoK):
        GtoK[1,i]=GtoK[1,0]+i*jdisGKsign
        GtoK[0,i]=int(GtoK[0,0]+i*jdisGKsign*idisGK/jdisGK)

idisKM = Mpoint[0]-Kpoint[0]
jdisKM = Mpoint[1]-Kpoint[1]
numKtoM = max(np.abs(idisKM),np.abs(jdisKM))
KtoM = np.zeros([2,numKtoM],dtype = int)
KtoM[0,0]=Kpoint[0]
KtoM[1,0]=Kpoint[1]
if np.abs(idisKM)>np.abs(jdisKM):
    idisKMsign = idisKM/np.abs(idisKM)
    for i in range(1,numKtoM):
        KtoM[0,i]=KtoM[0,0]+i*idisKMsign
        KtoM[1,i]=int(KtoM[1,0]+i*idisKMsign*jdisKM/idisKM)
else:
    jdisKMsign = jdisKM/np.abs(jdisKM)
    for i in range(1,numKtoM):
        KtoM[1,i]=KtoM[1,0]+i*jdisKMsign
        KtoM[0,i]=int(KtoM[0,0]+i*jdisKMsign*idisKM/jdisKM)
        
idisMG = Gpoint[0]-Mpoint[0]
jdisMG = Gpoint[1]-Mpoint[1]
numMtoG = max(np.abs(idisMG),np.abs(jdisMG))
MtoG = np.zeros([2,numMtoG+1],dtype = int)
MtoG[0,0]=Mpoint[0]
MtoG[1,0]=Mpoint[1]
if np.abs(idisMG)>np.abs(jdisMG):
    idisMGsign = idisMG/np.abs(idisMG)
    for i in range(1,numMtoG):
        MtoG[0,i]=MtoG[0,0]+i*idisMGsign
        MtoG[1,i]=int(MtoG[1,0]+i*idisMGsign*jdisMG/idisMG)
else:
    jdisMGsign = jdisMG/np.abs(jdisMG)
    for i in range(1,numMtoG):
        MtoG[1,i]=MtoG[1,0]+i*jdisMGsign
        MtoG[0,i]=int(MtoG[0,0]+i*jdisMGsign*idisMG/jdisMG)
MtoG[0,-1]=Gpoint[0]
MtoG[1,-1]=Gpoint[1]

# current correlation function along the path

Cll_GtoK = copy.copy(Cll)
Cll_GtoK = Cll_GtoK[GtoK[0,:],GtoK[1,:],:]
Cll_KtoM = copy.copy(Cll)
Cll_KtoM = Cll_KtoM[KtoM[0,:],KtoM[1,:],:]
Cll_MtoG = copy.copy(Cll)
Cll_MtoG = Cll_MtoG[MtoG[0,:],MtoG[1,:],:]
Cll_GKMG = np.concatenate((Cll_GtoK,Cll_KtoM,Cll_MtoG),axis=0)

Cltr_GtoK = copy.copy(Cltr)
Cltr_GtoK = Cltr_GtoK[GtoK[0,:],GtoK[1,:],:]
Cltr_KtoM = copy.copy(Cltr)
Cltr_KtoM = Cltr_KtoM[KtoM[0,:],KtoM[1,:],:]
Cltr_MtoG = copy.copy(Cltr)
Cltr_MtoG = Cltr_MtoG[MtoG[0,:],MtoG[1,:],:]
Cltr_GKMG = np.concatenate((Cltr_GtoK,Cltr_KtoM,Cltr_MtoG),axis=0)

Clti_GtoK = copy.copy(Clti)
Clti_GtoK = Clti_GtoK[GtoK[0,:],GtoK[1,:],:]
Clti_KtoM = copy.copy(Clti)
Clti_KtoM = Clti_KtoM[KtoM[0,:],KtoM[1,:],:]
Clti_MtoG = copy.copy(Clti)
Clti_MtoG = Clti_MtoG[MtoG[0,:],MtoG[1,:],:]
Clti_GKMG = np.concatenate((Clti_GtoK,Clti_KtoM,Clti_MtoG),axis=0)

Ctt_GtoK = copy.copy(Ctt)
Ctt_GtoK = Ctt_GtoK[GtoK[0,:],GtoK[1,:],:]
Ctt_KtoM = copy.copy(Ctt)
Ctt_KtoM = Ctt_KtoM[KtoM[0,:],KtoM[1,:],:]
Ctt_MtoG = copy.copy(Ctt)
Ctt_MtoG = Ctt_MtoG[MtoG[0,:],MtoG[1,:],:]
Ctt_GKMG = np.concatenate((Ctt_GtoK,Ctt_KtoM,Ctt_MtoG),axis=0)

Cfull_GtoK = copy.copy(Cfull)
Cfull_GtoK = Cfull_GtoK[GtoK[0,:],GtoK[1,:],:]
Cfull_KtoM = copy.copy(Cfull)
Cfull_KtoM = Cfull_KtoM[KtoM[0,:],KtoM[1,:],:]
Cfull_MtoG = copy.copy(Cfull)
Cfull_MtoG = Cfull_MtoG[MtoG[0,:],MtoG[1,:],:]
Cfull_GKMG = np.concatenate((Cfull_GtoK,Cfull_KtoM,Cfull_MtoG),axis=0)

qnum = np.size(Cll_GKMG,axis=0)
q_band,w_band = np.meshgrid(np.arange(qnum),wrange,indexing='ij')

fromthisw = 1 # not include w=0
uptothisw = -1 # high frequency w is not relevant
w_band = w_band[:,fromthisw:uptothisw] # only take w>0.
q_band = q_band[:,fromthisw:uptothisw]
Cll_band = Cll_GKMG[:,fromthisw:uptothisw]
Cltr_band = Cltr_GKMG[:,fromthisw:uptothisw]
Clti_band = Clti_GKMG[:,fromthisw:uptothisw]
Ctt_band = Ctt_GKMG[:,fromthisw:uptothisw]
Cfull_band = Cfull_GKMG[:,fromthisw:uptothisw]

# exclude the self-circling signal part for better fitting

Cll_noSC = copy.copy(Cll_band)
Cll_noSC[:,16:20]=0

Ctt_noSC = copy.copy(Ctt_band)
Ctt_noSC[:,16:20]=0

Cltr_noSC = copy.copy(Cltr_band)
Cltr_noSC[:,16:20]=0

Clti_noSC = copy.copy(Clti_band)
Clti_noSC[:,16:20]=0

Cfull_noSC = copy.copy(Cfull_band)
Cfull_noSC[:,16:20]=0.0

tickloc = [0,np.size(GtoK,axis=1),np.size(GtoK,axis=1)+np.size(KtoM,axis=1)-1,np.size(GtoK,axis=1)+np.size(KtoM,axis=1)+np.size(MtoG,axis=1)-1]

Csc_band = copy.copy(Cfull_band)
Cnsc_band = copy.copy(Cfull_noSC)

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(25,9))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

# first plot before cutting off the self-circling signal
img1 = ax1.contourf(q_band,w_band,Csc_band,levels=30)

cb1 = fig.colorbar(img1,ax=ax1)
ax1.set_ylabel('$\omega$',fontsize=30)
cb1.set_label('C',fontsize=30)
ax1.text(-10,0.31,'(a)',fontsize=30,va='top',ha='right')
ax1.tick_params(labelsize=20)
cb1.ax.tick_params(labelsize=20)
ax1.set_xticks(tickloc,["$\Gamma$","K","M","$\Gamma$"],fontsize=30)

# second plot after cutting off the self-circling signal
img2 = ax2.contourf(q_band,w_band,Cnsc_band,levels=30)

cb2 = fig.colorbar(img2,ax=ax2)
ax2.set_ylabel('$\omega$',fontsize=30)
cb2.set_label('C',fontsize=30)
ax2.text(-10,0.31,'(b)',fontsize=30,va='top',ha='right')
ax2.tick_params(labelsize=20)
cb2.ax.tick_params(labelsize=20)
ax2.set_xticks(tickloc,["$\Gamma$","K","M","$\Gamma$"],fontsize=30)

# to avoid the weird white lines appearing in pdf file of the plot
for c in img1.collections:
    c.set_edgecolor("face")

for c in img2.collections:
    c.set_edgecolor("face")

#plt.savefig('band_expt_cutcomp_test.pdf',dpi=200,bbox_inches='tight')
plt.show()
